In [37]:
# Install necessary libraries
!pip install transformers
!pip install --quiet ipywidgets
!pip install spacy
!python -m spacy download en_core_web_sm

# Import libraries
import spacy
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import random
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM, pipeline


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [38]:
# Metrics initialization
nlu_total = 0
nlu_correct = 0

user_model_estimations = []
user_model_actual = []

pre_test_scores = {}
post_test_scores = {}

user_satisfaction_scores = []


In [39]:
# Define intents
intents = ['ask_concept', 'ask_explanation', 'solve_problem', 'greeting', 'goodbye', 'unknown']

# Simple rule-based intent classifier
def classify_intent(user_input):
    user_input = user_input.lower()
    if any(greet in user_input for greet in ['hi', 'hello', 'hey']):
        return 'greeting'
    elif any(bye in user_input for bye in ['bye', 'goodbye', 'see you']):
        return 'goodbye'
    elif any(ask in user_input for ask in ['explain', 'what is', 'define']):
        return 'ask_explanation'
    elif any(solve in user_input for solve in ['solve', 'help me with', 'how to']):
        return 'solve_problem'
    elif any(concept in user_input for concept in concepts):
        return 'ask_concept'
    else:
        return 'unknown'


In [40]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define domain-specific entities and patterns
from spacy.matcher import PhraseMatcher

# Define educational concepts
concepts = ["algebra", "calculus", "statistics", "derivative", "integral", "equation", "probability"]
patterns = [nlp.make_doc(text) for text in concepts]

# Initialize PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)
matcher.add("CONCEPT", patterns)

def extract_entities(user_input):
    doc = nlp(user_input)
    matches = matcher(doc)
    entities = []
    for match_id, start, end in matches:
        span = doc[start:end]
        entities.append(span.text.lower())
    return entities


In [41]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

# Define the Bayesian model structure
model = BayesianModel()

# Add nodes (concepts)
concepts_capitalized = [concept.capitalize() for concept in concepts]
model.add_nodes_from(concepts_capitalized)

# Define the CPDs (Prior probabilities)
cpds = []
for concept in concepts_capitalized:
    cpd = TabularCPD(variable=concept, variable_card=2, values=[[0.5], [0.5]], state_names={concept: ['known', 'unknown']})
    cpds.append(cpd)

# Add CPDs to the model
model.add_cpds(*cpds)

# Check if the model is valid
assert model.check_model()


In [42]:
def update_knowledge_state(concept, learned=True):
    concept_cap = concept.capitalize()
    if concept_cap in concepts_capitalized:
        cpd = model.get_cpds(concept_cap)
        current_known_prob = cpd.values[0][0]
        if learned:
            new_known_prob = min(current_known_prob + 0.1, 1.0)
        else:
            new_known_prob = max(current_known_prob - 0.1, 0.0)
        new_unknown_prob = 1 - new_known_prob
        new_cpd = TabularCPD(variable=concept_cap, variable_card=2, values=[[new_known_prob], [new_unknown_prob]], state_names={concept_cap: ['known', 'unknown']})
        model.add_cpds(new_cpd)
        model.check_model()
        # Record estimation and actual value for evaluation
        user_model_estimations.append(new_known_prob)
        actual_value = 1.0 if actual_knowledge_state.get(concept.lower(), False) else 0.0
        user_model_actual.append(actual_value)


In [43]:
def generate_system_response(user_input):
    # NLU: Extract entities and classify intent
    entities = extract_entities(user_input)
    intent = classify_intent(user_input)

    # Evaluate NLU accuracy
    evaluate_nlu(user_input, intent)

    # Default concept
    concept = entities[0] if entities else "general mathematics"

    if intent == 'ask_concept':
        response_text = templates[intent].format(concept=concept)
    elif intent == 'ask_explanation':
        explanation = generate_explanation(concept)
        response_text = templates[intent].format(concept=concept, explanation=explanation)
        update_knowledge_state(concept, learned=True)
    elif intent == 'solve_problem':
        problem = generate_problem(concept)
        response_text = templates[intent].format(concept=concept, problem=problem)
        update_knowledge_state(concept, learned=True)
    elif intent == 'greeting':
        response_text = templates[intent]
    elif intent == 'goodbye':
        response_text = templates[intent]
    else:
        response_text = templates['unknown']

    system_response = {
        "text": response_text,
        "intent": intent,
        "concept": concept
    }

    return system_response


In [44]:
# Load GPT-Neo model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

# Define a pad token if not already present
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model_gptneo = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
model_gptneo.resize_token_embeddings(len(tokenizer))  # Resize embeddings if pad token was added


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 768)

In [45]:
def generate_explanation(concept):
    prompt = f"Explain the concept of {concept} in simple terms suitable for a beginner.\n\nExplanation:"
    inputs = tokenizer(prompt, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    outputs = model_gptneo.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=input_ids.shape[1] + 100,  # Limit to 100 tokens beyond input
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    explanation = generated_text[len(prompt):].strip()
    return explanation


In [46]:
def generate_problem(concept):
    prompt = f"Create a simple practice problem for a beginner in {concept}.\n\nProblem:"
    inputs = tokenizer(prompt, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    outputs = model_gptneo.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=input_ids.shape[1] + 50,  # Limit to 50 tokens beyond input
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    problem = generated_text[len(prompt):].strip()
    return problem


In [47]:
# Template-based responses
templates = {
    'ask_concept': "Would you like to learn about {concept}?",
    'ask_explanation': "Sure, here's an explanation of {concept}: {explanation}",
    'solve_problem': "Let's solve a problem in {concept}. {problem}",
    'greeting': "Hello! How can I assist you in your learning today?",
    'goodbye': "Goodbye! Happy learning!",
    'unknown': "I'm sorry, I didn't quite understand that. Could you please rephrase?"
}

def generate_system_response(user_input):
    # NLU: Extract entities and classify intent
    entities = extract_entities(user_input)
    intent = classify_intent(user_input)

    # Evaluate NLU accuracy
    evaluate_nlu(user_input, intent)

    # Default concept
    concept = entities[0] if entities else "general mathematics"

    if intent == 'ask_concept':
        response_text = templates[intent].format(concept=concept)
    elif intent == 'ask_explanation':
        explanation = generate_explanation(concept)
        response_text = templates[intent].format(concept=concept, explanation=explanation)
        update_knowledge_state(concept, learned=True)
    elif intent == 'solve_problem':
        problem = generate_problem(concept)
        response_text = templates[intent].format(concept=concept, problem=problem)
        update_knowledge_state(concept, learned=True)
    elif intent == 'greeting':
        response_text = templates[intent]
    elif intent == 'goodbye':
        response_text = templates[intent]
    else:
        response_text = templates['unknown']

    system_response = {
        "text": response_text,
        "intent": intent,
        "concept": concept
    }

    return system_response


In [48]:
def chat():
    # Create input and output widgets
    input_box = widgets.Textarea(
        placeholder='Type your message here...',
        description='User:',
        disabled=False,
        layout=widgets.Layout(width='600px', height='80px')
    )
    send_button = widgets.Button(description='Send')
    output_area = widgets.Output()
    satisfaction_slider = widgets.IntSlider(
        value=5,
        min=1,
        max=5,
        step=1,
        description='Satisfaction:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )

    conversation_history = []

    # Simulate pre-test score
    pre_test_score = random.randint(50, 70)
    user_id = 'user_1'  # For demonstration purposes
    pre_test_scores[user_id] = pre_test_score

    # Define what happens when the send button is clicked
    def on_send_button_clicked(b):
        with output_area:
            clear_output(wait=True)
            user_input = input_box.value
            print(f"User: {user_input}\n")
            system_response = generate_system_response(user_input)
            print(f"Assistant: {system_response['text']}\n")
            conversation_history.append({"user": user_input, "assistant": system_response['text']})
            input_box.value = ''
            # Display satisfaction slider
            display(satisfaction_slider)

    # Collect user satisfaction score when slider value changes
    def on_satisfaction_change(change):
        if change['type'] == 'change' and not change['new'] is None:
            user_satisfaction_scores.append(change['new'])
            print(f"Satisfaction score recorded: {change['new']}\n")
            satisfaction_slider.value = 5  # Reset slider

    satisfaction_slider.observe(on_satisfaction_change, names='value')
    send_button.on_click(on_send_button_clicked)

    # Display the chat interface
    display(widgets.VBox([input_box, send_button, output_area]))

    # Simulate post-test score after the conversation ends
    def simulate_post_test():
        # For simplicity, assume learning gains are proportional to interactions
        interactions = len(conversation_history)
        post_test_score = pre_test_scores[user_id] + interactions * 2  # Gain 2 points per interaction
        post_test_scores[user_id] = min(post_test_score, 100)  # Cap at 100%

    # Run the simulation after the chat
    import atexit
    atexit.register(simulate_post_test)


In [49]:
chat()
